<a href="https://colab.research.google.com/github/aymannc/facial-recognition-api/blob/master/web_service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask-ngrok
!pip install face_recognition
!pip install pathlib
!pip install Pillow
!pip install opencv-python
!pip  install flask-cors

     |████████████████████████████████| 100.2MB 101kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566172 sha256=df158f11df2b9ba303dc779994d8c19188b01851fe953bad2122794fe53d8781
  Stored in directory: /root/.cache/pip/wheels/d2/99/18/59c6c8f01e39810415c0e63f5bede7d83dfb0ffc039865465f
Successfully built face-recognition-models


In [2]:
import dlib.cuda as cuda
print(cuda.get_num_devices())
import gc
import os
import pickle
import time
from pathlib import Path

import cv2
import face_recognition
import numpy as np
from PIL import Image
from flask import Flask, jsonify, request, send_from_directory, Response
from flask_cors import CORS
from imutils import paths
from flask_ngrok import run_with_ngrok
from flask import Flask
import json
import requests
from IPython.display import Image as show_image
from google.colab import drive

drive.mount('/content/drive')


1
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!ls "/content/drive/My Drive/FaceRec Project/dataset"
!ls "/content/drive/My Drive/FaceRec Project/results"

 Achir	      Assimi	   Bourase   Ghomari	   Khattach    Redouani
 Ahriz	      Atibi	   Chafi     Habil	   Lablaoui    Sabar
'Ait Alla'   'Ayman NC'    Chihab    Haddadi	   Meslouhi    Sakhi
'Ait Daoud'   BabaKhouya   Chihi    'Haj Sallem'   Ouftou      Youssfi
 Amzil	      Badaoui	   Dadi      Hamman	   Oulahyane   Zedouti
 Annan	      Benmousa	   Faiq      KhaifAllah    Ounjim
 Anssari      Berrada	   Geuddi    Kharazi	   rat
1591216617.2455087.jpg	1591217238.2081256.jpg	1591221550.7420495.jpg
1591216823.0088308.jpg	1591217853.4258227.jpg	1591223328.4779892.jpg
1591216984.4768505.jpg	1591221268.2724173.jpg	1591223398.448467.jpg
1591217192.7457218.jpg	1591221506.0897892.jpg


In [0]:
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif'}
data_set_path="/content/drive/My Drive/FaceRec Project/dataset"
results_path="/content/drive/My Drive/FaceRec Project/results"
encodings_path="/content/drive/My Drive/FaceRec Project/encodings.pickle"
public_url = None
app = Flask(__name__, static_url_path='')
CORS(app)

run_with_ngrok(app)
data = {}
data_load_time = 0.0
number_of_requests = 0


In [0]:
@app.route('/uploads/<path:path>')
def download_file(path):
    return send_from_directory(data_set_path, path)


@app.route('/results/<path:path>')
def download_results_image(path):
    return send_from_directory(results_path, path)
@app.before_first_request
def get_ngrok_url():
  global public_url
  url = "http://localhost:4040/api/tunnels"
  res = requests.get(url)
  res_unicode = res.content.decode("utf-8")
  res_json = json.loads(res_unicode)
  
  public_url = res_json["tunnels"][1]["public_url"]+"/"
  return res_json["tunnels"][1]["public_url"]+"/"

def load_global_data():
  global data
  global data_load_time
  start_time = time.time()
  print("[INFO] loading data")
  data = pickle.loads(open(encodings_path, "rb").read())
  data_load_time = time.time() - start_time
  print(f"[INFO] Done loading data")


def allowed_file(filename):
  return '.' in filename and \
        filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS


In [0]:


@app.route('/upload_images', methods=['POST'])
def upload_file():
    username = request.form.get('username') or "default"
    if request.method == 'POST' and 'files' in request.files:
        link_list = []
        try:
            for file in request.files.getlist('files'):
                if file and allowed_file(file.filename):
                    base_path = os.path.join(data_set_path, username)
                    file_name = f"{username}_{time.time()}{file.filename}"
                    Path(base_path).mkdir(parents=True, exist_ok=True)
                    full_path = os.path.join(base_path, file_name)
                    file.save(full_path)
                    link_list.append(f"{public_url}{username}/{file_name}")
        except Exception as _:
            pass
        return jsonify({"file_path": link_list})

@app.route('/facial_recognition', methods=['GET', 'POST'])
def upload_image():
    # Check if a valid image file was uploaded
    if request.method == 'POST':
        if 'file' not in request.files:
            return Response(
                "No image uploaded!",
                status=404
            )

        file = request.files['file']

        if file.filename == '':
            return Response("Image error!", status=415)

        if file and allowed_file(file.filename):
            # The image file seems valid! Detect faces and return the result.
            return detect_faces_in_image(file)

    # If no valid image file was uploaded, show the file upload form:
    return '''
    <!doctype html>
    <title>Is this a picture of X?</title>
    <h1>Upload a picture !</h1>
    <form method="POST" enctype="multipart/form-data">
      <input type="file" name="file">
      <input type="submit" value="Upload">
    </form>
    '''


@app.route('/faces_encoding', methods=['GET', 'POST'])
def faces_encoding():
    successful = True
    error_message = None

    start_time = time.time()
    try:
        print("[INFO] quantifying faces...")
        # Example of output ['dataset\\anc\\Nait Cherif.jpg',...]
        imagePaths = list(paths.list_images(data_set_path))
        print(imagePaths)
        # initialize the list of known encodings and known names
        # print(imagePaths)
        knownEncodings = []
        knownNames = []

        for (i, imagePath) in enumerate(imagePaths):
            # extract the person name from the image path
            print(f"[INFO] processing image {i + 1}/{len(imagePaths)}")
            name = imagePath.split(os.path.sep)[-2]
            # load the input image and convert it from BGR (OpenCV ordering)
            # to dlib ordering (RGB)
            image = cv2.imread(imagePath)
            height, width, _ = image.shape
            rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # detect the (x, y)-coordinates of the bounding boxes
            # corresponding to each face in the input image
            boxes = face_recognition.face_locations(rgb, model="cnn")
            # compute the facial embedding for the face
            encodings = face_recognition.face_encodings(rgb, boxes)
            # loop over the encodings
            for encoding in encodings:
                # add each encoding + name to our set of known names and
                # encodings
                knownEncodings.append(encoding)
                knownNames.append(name)

        # dump the facial encodings + names to disk
        print("[INFO] serializing encodings...")
        i_data = {"encodings": knownEncodings, "names": knownNames}
        with open(encodings_path, "wb") as f:
            f.write(pickle.dumps(i_data))
    except Exception as e:
        successful = False
        error_message = e
        print(e)
    finally:
        return jsonify({
            "successful": successful,
            "error_message": str(error_message),
            "time_to_complete": time.time() - start_time
        })


def detect_faces_in_image(file_stream):
    load_global_data()
    start_time = time.time()
    print("[INFO] recognizing faces...")
    # Load the uploaded image file
    original_img = Image.open(file_stream).convert('RGB')
    img = np.array(original_img)
    boxes = face_recognition.face_locations(img, model="cnn")
    encodings = face_recognition.face_encodings(img, boxes)
    # Get face encodings for any faces in the uploaded image
    names = []

    for encoding in encodings:
        global data
        matches = face_recognition.compare_faces(
            data["encodings"], encoding, tolerance=0.5)
        name = "Unknown"
        if True in matches:
            # find the indexes of all matched faces then initialize a
            # dictionary to count the total number of times each face
            # was matched
            matchedIdxs = [i for (i, b) in enumerate(matches) if b]
            counts = {}
            # loop over the matched indexes and maintain a count for
            # each recognized face
            for i in matchedIdxs:
                name = data["names"][i]
                counts[name] = counts.get(name, 0) + 1
            # determine the recognized face with the largest number of
            # votes (note: in the event of an unlikely tie Python will
            # select first entry in the dictionary)
            name = max(counts, key=counts.get)
            print("[INFO]counts", counts)
            # update the list of names
        names.append(name)
    image = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    for (top, right, bottom, left), name in zip(boxes, names):
        # Draw a box around the face
        cv2.rectangle(image, (left - 20, top - 20),
                      (right + 20, bottom + 20), (255, 0, 0), 2)

        # Draw a label with a name below the face
        cv2.rectangle(image, (left - 20, bottom),
                      (right + 20, bottom + 20), (255, 0, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(image, name, (left - 20, bottom + 15),
                    font, 0.7, (255, 255, 255))

    # show the output image
    file_name = f"{time.time()}.jpg"
    file_path = f"{results_path}/{file_name}"
    cv2.imwrite(file_path, image)
    show_image(file_path)
    result = {
        "results_file_url": f"{public_url}results/{file_name}"
        "faces_found_in_image": names,
        "faces_load_time": data_load_time,
        "data_rec_time": time.time() - start_time,
    }
    return jsonify(result)


In [0]:
if __name__ == "__main__":
    # TODO : fix aout of memory clean code
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://7129947fd486.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [03/Jun/2020 22:37:39] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [03/Jun/2020 22:37:39] "GET /robots.txt HTTP/1.1" 404 -
127.0.0.1 - - [03/Jun/2020 22:37:41] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [03/Jun/2020 22:38:03] "GET /facial_recognition HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2020 22:38:10] "GET /facial_recognition HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2020 22:38:11] "GET /favicon.ico HTTP/1.1" 404 -


[INFO] loading data
[INFO] Done loading data
[INFO] recognizing faces...


...

127.0.0.1 - - [03/Jun/2020 22:44:26] "POST /facial_recognition HTTP/1.1" 200 -


[INFO]counts {'Ouftou': 4, 'Bourase': 1, 'BabaKhouya': 2}


127.0.0.1 - - [03/Jun/2020 22:44:29] "GET /facial_recognition HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2020 22:44:37] "GET /results/1591224266.430754.jpg HTTP/1.1" 200 -


[INFO] loading data
[INFO] Done loading data
[INFO] recognizing faces...


127.0.0.1 - - [03/Jun/2020 22:53:20] "POST /facial_recognition HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2020 22:53:23] "GET /results/1591224800.1201506.jpg HTTP/1.1" 200 -
